In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from flask import Flask, render_template, request

In [2]:
df = pd.read_excel("dataset.xlsx")

=== PREPROCESSING TEXT ===

In [3]:
!pip install Sastrawi

Defaulting to user installation because normal site-packages is not writeable


In [4]:
#https://chat.openai.com/share/6da81ec8-3cd1-48d6-9ee1-e29202cfe7c5

In [5]:
import string
import re
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Diputra_W\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def case_folding(sentence):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # Emojis in the first range
        u"\U0001F300-\U0001F5FF"  # Emojis in the second range
        u"\U0001F680-\U0001F6FF"  # Emojis in the third range
        u"\U0001F700-\U0001F77F"  # Emojis in the fourth range
        u"\U0001F780-\U0001F7FF"  # Emojis in the fifth range
        u"\U0001F800-\U0001F8FF"  # Emojis in the sixth range
        u"\U0001F900-\U0001F9FF"  # Emojis in the seventh range
        u"\U0001FA00-\U0001FA6F"  # Emojis in the eighth range
        u"\U0001FA70-\U0001FAFF"  # Emojis in the ninth range
        u"\U0001F004-\U0001F0CF"  # Emojis in the tenth range
        "]+", flags=re.UNICODE)

    sentence = emoji_pattern.sub(r'', sentence)
    sentence = sentence.translate(str.maketrans("","", string.punctuation)).lower()
    sentence = re.sub(r"\d+", "", sentence)
    sentence = sentence.replace("/", " ")
    return sentence


In [8]:
def load_abbreviation_file(file_path):
    try:
        with open(file_path, "r") as file:
            abbreviations = json.load(file)
        return abbreviations
    except FileExistsError:
        print(f"File not found {file_path}")
        return {}

#Reading the abbreviation file path for preprocessing
file_path = "abbreviation_file.txt"
abbreviation_file = load_abbreviation_file(file_path)

def normalize_text(sentence):
    words = sentence.lower().split()
    words_normalized = []
    for word in words:
        for full_form, abbreviations in abbreviation_file.items():
            if word.lower() in abbreviations:
                words_normalized.append(full_form)
                break
        else:
            words_normalized.append(word)
    return " ".join(words_normalized)

In [9]:
def stopwords_removal(sentence):
    tokens = word_tokenize(sentence)
    liststopwords =  set(stopwords.words('indonesian'))

    custom_stopwords_file = "more_stopwords.txt"

    custom_stopwords = set()
    with open(custom_stopwords_file, "r") as file:
        for line in file:
            custom_stopwords.add(line.strip())

    combined_stopwords = liststopwords.union(custom_stopwords)

    with open(custom_stopwords_file, "w") as file:
        for word in combined_stopwords:
            file.write(word + "\n")

def remove_custom_stopwords(sentence, custom_stopwords_file):
    custom_stopwords = set()
    with open(custom_stopwords_file, 'r') as file:
        for line in file:
            custom_stopwords.add(line.strip())

    words = word_tokenize(sentence)

    filtered_words = [word for word in words if word.lower() not in custom_stopwords]

    cleaned_text = ' '.join(filtered_words)

    return cleaned_text

In [10]:
def stemming_text(sentence):
    factory = StemmerFactory()
    Stemmer = factory.create_stemmer()

    sentence = Stemmer.stem(sentence)
    return sentence

In [11]:
df['case_folding'] = df['review_text'].apply(case_folding)
df['normalized_text'] = df['case_folding'].apply(normalize_text)
df['stopword_removed'] = df['normalized_text'].apply(lambda x: remove_custom_stopwords(x, 'more_stopwords.txt'))
df['stemmed_text'] = df['stopword_removed'].apply(stemming_text)
pd.set_option('display.max_colwidth', None)
df.head(1)

,author_title,review_rating,review_text,case_folding,normalized_text,stopword_removed,stemmed_text
0,Felix zhang,5,"Urus KTP hilang cukup surat keterangan hilang dari kepolisian dan fotokopi KK\nAntrian sekitar 1 jam, proses cetak hanya sekitar 2 menit",urus ktp hilang cukup surat keterangan hilang dari kepolisian dan fotokopi kk\nantrian sekitar jam proses cetak hanya sekitar menit,urus kartu tanda penduduk hilang cukup surat keterangan hilang dari kepolisian dan fotokopi kartu keluarga antrian sekitar jam proses cetak hanya sekitar menit,urus kartu tanda penduduk hilang surat keterangan hilang kepolisian fotokopi kartu keluarga antrian jam proses cetak menit,urus kartu tanda duduk hilang surat terang hilang polisi fotokopi kartu keluarga antri jam proses cetak menit


In [12]:
dfnew = df[["stemmed_text", "review_rating"]]
dfnew.head()

,stemmed_text,review_rating
0,urus kartu tanda duduk hilang surat terang hilang polisi fotokopi kartu keluarga antri jam proses cetak menit,5
1,urus akta nikah online taring proses cetak arah kantor informasi terima email cetak akta paket kartu tanda duduk kartu keluarga terbit admin taring respon tanggap cepat respect admin suasana dukcapil loket layan cepat ramah tunggu nyaman antri ramai paham alur mudah cepat,5
2,layan mana buat suratsurat mudah sistem online bulakbalik penuh dokumen langsung cetak mandiri bawa ojek online tugas ramah bimbing sulit moga layan tahan tingkat terimakasih,5
3,gaji staf staf dukcapil kerja buru buru pulang jam salah salah cetak kartu tanda duduk kartu keluarga status agama kartu tanda duduk ganti buru buru pulang kerja beres,1
4,proses cepat online riweh bingung tinggal admin layan ramah ya beda sangtta yaa sih ter the best pokok,5


In [13]:
import tensorflow
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras import models

In [14]:
# Custom label mapping
label_mapping = {1: 'Negative', 2: 'Negative', 3: 'Neutral', 4: 'Positive', 5: 'Positive'}

# Create a new DataFrame with only the relevant columns
df_encoded = pd.DataFrame()
df_encoded['stemmed_text'] = dfnew['stemmed_text']

# Map labels to the custom categories
df_encoded['sentiment_category'] = dfnew['review_rating'].map(label_mapping)

# Handle any remaining NaN values (replace with a default value, e.g., 'Neutral')
df_encoded['sentiment_category'].fillna('Neutral', inplace=True)

# Now you can proceed with label encoding
label_encoder = LabelEncoder()
df_encoded['encoded_label'] = label_encoder.fit_transform(df_encoded['sentiment_category'])

# Print the new DataFrame to inspect
df_encoded.head()

,stemmed_text,sentiment_category,encoded_label
0,urus kartu tanda duduk hilang surat terang hilang polisi fotokopi kartu keluarga antri jam proses cetak menit,Positive,2
1,urus akta nikah online taring proses cetak arah kantor informasi terima email cetak akta paket kartu tanda duduk kartu keluarga terbit admin taring respon tanggap cepat respect admin suasana dukcapil loket layan cepat ramah tunggu nyaman antri ramai paham alur mudah cepat,Positive,2
2,layan mana buat suratsurat mudah sistem online bulakbalik penuh dokumen langsung cetak mandiri bawa ojek online tugas ramah bimbing sulit moga layan tahan tingkat terimakasih,Positive,2
3,gaji staf staf dukcapil kerja buru buru pulang jam salah salah cetak kartu tanda duduk kartu keluarga status agama kartu tanda duduk ganti buru buru pulang kerja beres,Negative,0
4,proses cepat online riweh bingung tinggal admin layan ramah ya beda sangtta yaa sih ter the best pokok,Positive,2


In [15]:
X_train, X_test, y_train, y_test = train_test_split(df_encoded['stemmed_text'], df_encoded['encoded_label'], test_size=0.2, random_state=42)

In [16]:
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 100  # Adjust based on your dataset
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

In [17]:
# Assuming you have a DataFrame df_encoded with 'text' and 'encoded_label' columns
# One-hot encode the labels
df_encoded['encoded_label'] = to_categorical(df_encoded['encoded_label'])
df_encoded.head()

,stemmed_text,sentiment_category,encoded_label
0,urus kartu tanda duduk hilang surat terang hilang polisi fotokopi kartu keluarga antri jam proses cetak menit,Positive,0.0
1,urus akta nikah online taring proses cetak arah kantor informasi terima email cetak akta paket kartu tanda duduk kartu keluarga terbit admin taring respon tanggap cepat respect admin suasana dukcapil loket layan cepat ramah tunggu nyaman antri ramai paham alur mudah cepat,Positive,0.0
2,layan mana buat suratsurat mudah sistem online bulakbalik penuh dokumen langsung cetak mandiri bawa ojek online tugas ramah bimbing sulit moga layan tahan tingkat terimakasih,Positive,0.0
3,gaji staf staf dukcapil kerja buru buru pulang jam salah salah cetak kartu tanda duduk kartu keluarga status agama kartu tanda duduk ganti buru buru pulang kerja beres,Negative,1.0
4,proses cepat online riweh bingung tinggal admin layan ramah ya beda sangtta yaa sih ter the best pokok,Positive,0.0


In [18]:
# Build a simple neural network
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=3, activation='softmax'))  # 3 output units for three categories

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, to_categorical(y_train), epochs=15, validation_data=(X_test_padded, to_categorical(y_test)))


Epoch 1/15
21/21 [==============================] - 2s 41ms/step - loss: 0.6295 - accuracy: 0.7581 - val_loss: 0.5369 - val_accuracy: 0.7607
Epoch 2/15
21/21 [==============================] - 1s 26ms/step - loss: 0.4792 - accuracy: 0.8274 - val_loss: 0.4630 - val_accuracy: 0.8589
Epoch 3/15
21/21 [==============================] - 1s 26ms/step - loss: 0.3371 - accuracy: 0.9060 - val_loss: 0.4021 - val_accuracy: 0.8650
Epoch 4/15
21/21 [==============================] - 1s 26ms/step - loss: 0.2149 - accuracy: 0.9291 - val_loss: 0.3620 - val_accuracy: 0.8773
Epoch 5/15
21/21 [==============================] - 1s 25ms/step - loss: 0.1419 - accuracy: 0.9615 - val_loss: 0.3473 - val_accuracy: 0.8834
Epoch 6/15
21/21 [==============================] - 1s 32ms/step - loss: 0.1016 - accuracy: 0.9692 - val_loss: 0.3364 - val_accuracy: 0.8896
Epoch 7/15
21/21 [==============================] - 1s 29ms/step - loss: 0.0776 - accuracy: 0.9769 - val_loss: 0.3339 - val_accuracy: 0.8834
Epoch 8/15
21

In [19]:
#Save the model and tokenizer
model.save('sentiment_model.h5')
tokenizer_config = tokenizer.get_config()
tokenizer_config['num_words'] = max_words
with open('tokenizer_config.json', 'w') as config_file:
    config_file.write(json.dumps(tokenizer_config))

In [20]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# ...

# Load the model and tokenizer
model = models.load_model('sentiment_model.h5')
with open('tokenizer_config.json') as config_file:
    config = json.load(config_file)
    max_words = config['num_words']
    tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
    tokenizer.word_index = json.loads(config['word_index'])
    tokenizer.index_word = {str(i): word for word, i in tokenizer.word_index.items()}

# ...


In [21]:
app = Flask(__name__)
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        text = request.form['text']
        result = perform_sentiment_analysis(text)
        return render_template('index.html', result=result, text=text)


In [22]:
def perform_sentiment_analysis(text):
    # Preprocess the input text
    preprocessed_text = preprocess_text(text)

    # Tokenize and pad the sequence
    max_length = 100  # Adjust based on your dataset
    text_seq = tokenizer.texts_to_sequences([preprocessed_text])
    text_padded = pad_sequences(text_seq, maxlen=max_length, padding='post')

    # Make prediction
    prediction = model.predict(text_padded)
    sentiment = 'Positive' if prediction > 0.5 else 'Negative'

    return sentiment


In [23]:
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Diputra_W\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
import nltk
import json  # Import the json module
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

app = Flask(__name__)

# Load the data from the Excel file
df = pd.read_excel('your_file.xlsx')  # Replace 'your_file.xlsx' with the actual path to your Excel file

# Preprocess the data
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Removing punctuation
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])

    # Removing stopwords

    return text

df['text'] = df['text'].apply(preprocess_text)

# Encode labels to numerical values
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Tokenize and pad the sequences
max_words = 10000  # Adjust based on your dataset
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 100  # Adjust based on your dataset
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Build a simple neural network
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, validation_data=(X_test_padded, y_test))

# Save the model and tokenizer
model.save('sentiment_model.h5')
tokenizer_config = tokenizer.get_config()
tokenizer_config['num_words'] = max_words
with open('tokenizer_config.json', 'w') as config_file:
    config_file.write(json.dumps(tokenizer_config))

# Load the model and tokenizer
model = load_model('sentiment_model.h5')
with open('tokenizer_config.json') as config_file:
    config = json.load(config_file)
    tokenizer = Tokenizer.from_config(config)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        text = request.form['text']
        result = perform_sentiment_analysis(text)
        return render_template('index.html', result=result, text=text)

def perform_sentiment_analysis(text):
    # Preprocess the input text
    preprocessed_text = preprocess_text(text)

    # Tokenize and pad the sequence
    max_length = 100  # Adjust based on your dataset
    text_seq = tokenizer.texts_to_sequences([preprocessed_text])
    text_padded = pad_sequences(text_seq, maxlen=max_length, padding='post')

    # Make prediction
    prediction = model.predict(text_padded)
    sentiment = 'Positive' if prediction > 0.5 else 'Negative'

    return sentiment

if __name__ == '__main__':
    app.run(debug=True)
